<a href="https://colab.research.google.com/github/nedokormysh/GB_NLP_intro/blob/lesson7/NLP_intro_hw_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Практическое задание к уроку 7 по теме "Сверточные нейронные сети для анализа текста".
Взять датасет отзывов за лето (из архива с материалами или предыдущего занятия)
Обучить conv сеть в двух вариантах:

Инициализировать Embedding предобученными векторами, взять к примеру с https://rusvectores.org/ru/.

Инициализировать слой Embedding по умолчанию (ну то есть ничего не делать с весами)

Сравнить две архитектуры с предобученными весами и когда Embedding обучается сразу со всей сетью, что получилось лучше.

Загрузим библиотеки и датасет:

In [47]:
%matplotlib inline

from collections import Counter
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# from pymorphy2 import MorphAnalyzer
# from razdel import tokenize
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MaxAbsScaler
# from stop_words import get_stop_words
from string import punctuation
import torch
from torch import nn
# from torchinfo import summary
from tqdm import tqdm
# from transformers import AutoTokenizer, AutoModelForMaskedLM

In [48]:
!pip install torchinfo -q

In [49]:
from torchinfo import summary

In [50]:
df = pd.read_excel('https://github.com/nedokormysh/GB_NLP_intro/blob/lesson7/%D0%BE%D1%82%D0%B7%D1%8B%D0%B2%D1%8B%20%D0%B7%D0%B0%20%D0%BB%D0%B5%D1%82%D0%BE.xls?raw=true')
df

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14
...,...,...,...
20654,1,"Ну и шляпа,с роот правами бесполезная прога,ра...",2017-06-01
20655,5,Ок,2017-06-01
20656,4,Доволен,2017-06-01
20657,1,"Песопаснасть, рут ни нужын",2017-06-01


In [51]:
df.isna().sum()

Rating     0
Content    3
Date       0
dtype: int64

In [52]:
df = df.dropna()

Удалили пропущенные значения.

In [53]:
!pip install pymorphy2
!pip install stop_words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [54]:
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

In [55]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df['text'] = df['Content'].apply(preprocess_text)

<ipython-input-55-12a05b0b01cb>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['Content'].apply(preprocess_text)


In [56]:
df.Rating.value_counts()

5    14584
1     2276
4     2137
3      911
2      748
Name: Rating, dtype: int64

Видим несбалансированность классов. Потребуется стратификация.

In [57]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.text, df.Rating, test_size=0.3, random_state=7575, stratify=df.Rating)

In [58]:
train_corpus = " ".join(X_train)
train_corpus = train_corpus.lower()

In [59]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

tokens = word_tokenize(train_corpus)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Отфильтруем данные

и соберём в корпус N наиболее частых токенов

In [60]:
tokens_filtered = [word for word in tokens if word.isalnum()]

In [61]:
max_words = 10000
max_len = 100
num_classes = 6

# Training
epochs = 5
batch_size = 512
print_batch_n = 100

In [62]:
from nltk.probability import FreqDist
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [63]:
tokens_filtered_top[:10]

['приложение',
 'удобно',
 'работать',
 'удобный',
 'отлично',
 'нравиться',
 'отличный',
 'хороший',
 'телефон',
 'супер']

In [64]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [65]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(str(text).lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [66]:
X_train = np.asarray([text_to_sequence(text, max_len) for text in X_train], dtype=np.int32)
X_test = np.asarray([text_to_sequence(text, max_len) for text in X_test], dtype=np.int32)

In [67]:
# X_train

# Keras model

In [68]:
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
# from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping  

In [69]:
import keras
from keras.utils import np_utils

y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
y_test = keras.utils.np_utils.to_categorical(y_test, num_classes)

## По умолчанию

In [70]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [72]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [77]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    # callbacks=[tensorboard, early_stopping]
                    )

Epoch 1/5
26/26 [==============================] - 7s 271ms/step - loss: 0.5847 - accuracy: 0.8070 - val_loss: 0.7266 - val_accuracy: 0.7877
Epoch 2/5
26/26 [==============================] - 7s 252ms/step - loss: 0.5406 - accuracy: 0.8196 - val_loss: 0.7311 - val_accuracy: 0.7863
Epoch 3/5
26/26 [==============================] - 7s 276ms/step - loss: 0.4981 - accuracy: 0.8389 - val_loss: 0.7290 - val_accuracy: 0.7787
Epoch 4/5
26/26 [==============================] - 6s 243ms/step - loss: 0.4571 - accuracy: 0.8570 - val_loss: 0.7679 - val_accuracy: 0.7718
Epoch 5/5
26/26 [==============================] - 7s 274ms/step - loss: 0.4180 - accuracy: 0.8704 - val_loss: 0.7768 - val_accuracy: 0.7732


In [82]:
preds = model.predict(X_test)

194/194 [==============================] - 1s 6ms/step


In [80]:
preds

array([[9.1270008e-04, 3.0440374e-06, 2.2835615e-05, 2.3443565e-05,
        5.7279397e-02, 9.4175863e-01],
       [2.0677447e-02, 6.0056645e-01, 1.7829710e-01, 1.5778832e-01,
        2.8752541e-02, 1.3918195e-02],
       [1.3027743e-03, 5.8053952e-06, 3.4477947e-05, 2.4008415e-05,
        8.1105992e-02, 9.1752708e-01],
       ...,
       [2.2883108e-02, 1.5780651e-03, 2.8517176e-03, 1.3150411e-03,
        2.7915791e-01, 6.9221413e-01],
       [1.4631745e-03, 6.7142314e-06, 5.2458639e-05, 9.9021985e-05,
        4.7788378e-02, 9.5059025e-01],
       [6.4402847e-03, 8.3079624e-01, 1.0785139e-01, 4.6620980e-02,
        6.6984082e-03, 1.5927048e-03]], dtype=float32)

In [83]:
preds = [np.argmax(num) for num in preds]
# preds

In [84]:
from sklearn.metrics import classification_report

print(classification_report([np.argmax(l) for l in y_test], preds))

              precision    recall  f1-score   support

           1       0.50      0.67      0.58       683
           2       0.00      0.00      0.00       225
           3       0.17      0.15      0.16       273
           4       0.30      0.11      0.16       641
           5       0.86      0.94      0.90      4375

    accuracy                           0.75      6197
   macro avg       0.36      0.38      0.36      6197
weighted avg       0.70      0.75      0.72      6197



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [85]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 1s 84ms/step - loss: 0.8181 - accuracy: 0.7547


Test score: 0.8180538415908813
Test accuracy: 0.7547200322151184


## С предобученными векторами

In [86]:
# Загрузка предобученной модели

!wget https://rusvectores.org/static/models/rusvectores4/taiga/taiga_upos_skipgram_300_2_2018.vec.gz

!gunzip taiga_upos_skipgram_300_2_2018.vec.gz

--2023-03-16 10:40:30--  https://rusvectores.org/static/models/rusvectores4/taiga/taiga_upos_skipgram_300_2_2018.vec.gz
Resolving rusvectores.org (rusvectores.org)... 172.104.228.108
Connecting to rusvectores.org (rusvectores.org)|172.104.228.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 346385366 (330M) [application/x-gzip]
Saving to: ‘taiga_upos_skipgram_300_2_2018.vec.gz’

taiga_upos_skipgram 100%[===================>] 330.34M  29.5MB/s    in 11s     

2023-03-16 10:40:42 (29.2 MB/s) - ‘taiga_upos_skipgram_300_2_2018.vec.gz’ saved [346385366/346385366]



In [87]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['text'])
vocab_size = len(tokenizer.word_index) + 1

In [88]:
embeddings_index = {}
with open('taiga_upos_skipgram_300_2_2018.vec') as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        word = word.split('_')[0]
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 209666 word vectors.


In [89]:
embedding_dim = 300
hits = 0
missed = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        missed += 1

print("Converted %d words (%d missed)" % (hits, missed))

Converted 5612 words (7872 missed)


In [90]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=300, input_length=max_len, weights=[embedding_matrix],
    trainable=False))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [91]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [92]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    # callbacks=[tensorboard, early_stopping],
                    )

Epoch 1/5
26/26 [==============================] - 11s 400ms/step - loss: 1.5664 - accuracy: 0.6526 - val_loss: 1.3759 - val_accuracy: 0.6957
Epoch 2/5
26/26 [==============================] - 11s 429ms/step - loss: 1.2386 - accuracy: 0.7072 - val_loss: 1.1196 - val_accuracy: 0.6957
Epoch 3/5
26/26 [==============================] - 14s 547ms/step - loss: 1.0625 - accuracy: 0.7072 - val_loss: 1.0004 - val_accuracy: 0.6957
Epoch 4/5
26/26 [==============================] - 10s 396ms/step - loss: 0.9657 - accuracy: 0.7137 - val_loss: 0.9307 - val_accuracy: 0.7268
Epoch 5/5
26/26 [==============================] - 10s 398ms/step - loss: 0.8974 - accuracy: 0.7390 - val_loss: 0.8798 - val_accuracy: 0.7400


In [93]:
preds = model.predict(X_test)
preds = [np.argmax(num) for num in preds]
print(classification_report([np.argmax(l) for l in y_test], preds))

194/194 [==============================] - 2s 11ms/step
              precision    recall  f1-score   support

           1       0.42      0.46      0.44       683
           2       0.00      0.00      0.00       225
           3       0.00      0.00      0.00       273
           4       0.65      0.02      0.04       641
           5       0.78      0.97      0.87      4375

    accuracy                           0.74      6197
   macro avg       0.37      0.29      0.27      6197
weighted avg       0.67      0.74      0.67      6197



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [94]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 2s 148ms/step - loss: 0.8971 - accuracy: 0.7394


Test score: 0.897142767906189
Test accuracy: 0.7393900156021118


В многоклассовой классификации с разбалансированными классами лучше опираться не на accuracy, а на f1-score. Можно наблюдать, что модель по этой метрике фактически хорошо определяет только класс 5. И с предобученной сетью качество падает, вероятно из-за отсутствия эмбедингов для слов из нашего набора.